In [1]:
pip install numpy pandas tensorflow keras matplotlib tkinter pillow


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
import tkinter as tk
from tkinter import filedialog
from tkinter import Label
from PIL import Image, ImageTk

In [34]:
train_dir = r"C:\Users\guest1\Downloads\Animals_Data-20240713T102247Z-001\Animals_Data\Test"
validation_dir =r"C:\Users\guest1\Downloads\Animals_Data-20240713T102247Z-001\Animals_Data\Test"

In [37]:
# Hyperparameters
num_classes = 26
image_size = (299, 299)
batch_size = 32
learning_rate = 0.001
epochs = 5

In [39]:
# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

In [41]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1303 images belonging to 26 classes.
Found 1303 images belonging to 26 classes.


In [43]:
# Load the base model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))


In [44]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [47]:

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

In [49]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [51]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])


In [53]:
# Calculate steps per epoch and validation steps
steps_per_epoch = train_generator.samples // batch_size
validation_steps = valid_generator.samples // batch_size

In [55]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=validation_steps
)




Epoch 1/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 175s 4s/step - accuracy: 0.6560 - loss: 1.4243 - val_accuracy: 0.9719 - val_loss: 0.1111
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 1.0000 - loss: 0.0315 - val_accuracy: 1.0000 - val_loss: 0.0094
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 166s 4s/step - accuracy: 0.9689 - loss: 0.1341 - val_accuracy: 0.9937 - val_loss: 0.0380
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.9688 - loss: 0.0638 - val_accuracy: 1.0000 - val_loss: 0.0478
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - accuracy: 0.9895 - loss: 0.0506 - val_accuracy: 0.9930 - val_loss: 0.0275


In [56]:
# Save the model
model.save("inceptionv3_model.keras")

In [57]:
# Load the model
model = tf.keras.models.load_model('inceptionv3_model.keras')

In [58]:
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])


In [59]:
# Class labels
class_labels = ['Bear', 'Cat', 'Chicken', 'Dog', 'Duck', 'Elephant', 'Fish', 'Fox', 'Frog', 'Giraffe', 'Hippopotamus', 'Horse', 'Kangaroo', 'Koala', 'Lion', 'Monkey', 'Panda', 'Peacock', 'Penguin','Raccon','Rhinoceros','sheep','Snake','Tiger','Whale','Zebra']


In [60]:

# Define functions for image preprocessing and prediction
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((299, 299))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

In [61]:

def predict_animal(image_path):
  processed_image=preprocess_image(image_path)
  predictions=model.predict(processed_image)
  predicted_label=class_labels[np.argmax(predictions)]
  confidence=np.max(predictions)
  return predicted_label,confidence

'''def predict_animal(image_path):
    processed_image = preprocess_image(image_path)
    predictions = model.predict(processed_image)
    
    print(f"Predictions: {predictions}")  # Debugging statement
    print(f"Predictions shape: {predictions.shape}")  # Debugging statement
    print(f"Class labels: {class_labels}")  # Debugging statement

    if len(predictions[0]) != len(class_labels):
        raise ValueError(f"Mismatch between number of predictions ({len(predictions[0])}) and class labels ({len(class_labels)}).")
    
    predicted_label = class_labels[np.argmax(predictions)]
    confidence = np.max(predictions)
    return predicted_label, confidence'''

'def predict_animal(image_path):\n    processed_image = preprocess_image(image_path)\n    predictions = model.predict(processed_image)\n    \n    print(f"Predictions: {predictions}")  # Debugging statement\n    print(f"Predictions shape: {predictions.shape}")  # Debugging statement\n    print(f"Class labels: {class_labels}")  # Debugging statement\n\n    if len(predictions[0]) != len(class_labels):\n        raise ValueError(f"Mismatch between number of predictions ({len(predictions[0])}) and class labels ({len(class_labels)}).")\n    \n    predicted_label = class_labels[np.argmax(predictions)]\n    confidence = np.max(predictions)\n    return predicted_label, confidence'

In [65]:
'''import numpy as np

import matplotlib.pyplot as plt


uploaded_file_path=r"C:\Users\guest1\Downloads\Animals_Data-20240713T102247Z-001\Animals_Data\Test\Elephant\loxodonta-africana_40_37317fb0.jpg"
prediction,confidence=predict_animal(uploaded_file_path)
print(confidence)
img=tf.keras.utils.load_img(uploaded_file_path)
plt.imshow(img)
plt.axis('off')
if (confidence<0.80):

    print("Data could not be found.Sorry for the inconvienence")

else:
    plt.title(f'Prediction:{prediction}\nConfidence:{confidence:.2%}')
    plt.show()'''



# GUI for image upload and prediction
def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        prediction, confidence = predict_animal(file_path)
        img = Image.open(file_path)
        img = img.resize((299, 299))
        img = ImageTk.PhotoImage(img)
        img_label.config(image=img)
        img_label.image = img
        if confidence < 0.80:
            result_label.config(text="Data could not be found. Sorry for the inconvenience.")
        else:
            result_label.config(text=f"Prediction: {prediction}\nConfidence: {confidence:.2%}")



SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 77-78: truncated \UXXXXXXXX escape (2103945860.py, line 1)

In [67]:
# GUI for image upload and prediction
def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        prediction, confidence = predict_animal(file_path)
        img = Image.open(file_path)
        img = img.resize((299, 299))
        img = ImageTk.PhotoImage(img)
        img_label.config(image=img)
        img_label.image = img
        if confidence < 0.80:
            result_label.config(text="Data could not be found. Sorry for the inconvenience.")
        else:
            result_label.config(text=f"Prediction: {prediction}\nConfidence: {confidence:.2%}")



In [74]:
root = tk.Tk()
root.title("Animal Classifier")
root.geometry("800x600")  # Set window size to 800x600

root.configure(bg="lightblue")  # Set background color

welcome_label = tk.Label(root, text="Welcome to the Animal Classifier!", font=("Helvetica", 16), bg="lightblue")
welcome_label.pack(pady=20)

instructions_label = tk.Label(root, text="Please upload an image to classify.", font=("Helvetica", 12), bg="lightblue")
instructions_label.pack(pady=10)


In [76]:
top_frame = tk.Frame(root, bg="lightblue")
top_frame.pack(pady=20)

bottom_frame = tk.Frame(root, bg="lightblue")
bottom_frame.pack(pady=20)

upload_button = tk.Button(top_frame, text="Upload Image", command=upload_image)
upload_button.pack()

img_label = tk.Label(bottom_frame)
img_label.pack()

result_label = tk.Label(bottom_frame, text="", bg="lightblue")
result_label.pack()

root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
